# IBM Data Science Professional Certificate - Capstone Final Project - Oscar Antunes
##### 
### Analysis of Düsseldorf boroughs, Germany
##### 
#### Webscraping for  Boroughs, Houses available, Housing prices, Habitants, Venues
##### 


In [1]:
# Import required libraries for scrapping
import urllib.request
from bs4 import BeautifulSoup
import smtplib
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests


import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Regular Expressions to collect the addresses
import re

# Get the same number of District cells as Boroughs
from itertools import chain

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

# Class object has been created to store the credentials for safekeeping
#import credentials_oa_outlk

print('Libraries imported.')

Libraries imported.


# 
## Getting the Boroughs

In [2]:
url_wiki = 'https://de.wikipedia.org/wiki/Liste_der_Stadtbezirke_von_D%C3%BCsseldorf'

page = requests.get(url_wiki)
# Parsing through the URL
soup = BeautifulSoup(page.content, 'html.parser')
print("Data parsed.")

# Identify the table to scrap
right_table = soup.find('table', class_ = 'wikitable sortable')
print("Table located.")

Data parsed.
Table located.


# 
## Assigning Districts and Boroughs to DataFrame

In [3]:
# SET THE DATAFRAME COLUMNS || CREATE EMPTY DATAFRAME WITH THE SAME STRUCTURE OF THE ORIGINAL
column_names = ['District','Borough','Fläche','Einwohner','Bewölkerungsdichte','Bezirkvorsteher','K','Karte']
df = pd.DataFrame(columns = column_names)

#####################################################
# LOAD OF THE DATAFRAME || CLEAR TO THE 2 COLUMNS NEEDEED
for tr_cell in right_table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.rstrip())
        
    if len(row_data)>0:
        df.loc[len(df)] = row_data
        
df = df[['District', 'Borough']]
print(df.head(2),'\n')

#####################################################
# REMOVE NUMERIC DIGITS FROM THE BOROUGHS
df.set_index(['District', 'Borough'])  
df = df.replace(regex=r'[0-9]+', value=',')

# As the numeric digit has been removed from the District Column, 
# we need to add the number of each district as per the current index 
district_numb = ['1','2','3','4','5','6','7','8','9','10']
df['district_numb'] = district_numb

print(df.head(2))

           District                                            Borough
0  Stadtbezirk 1[1]  011 Altstadt012 Carlstadt013 Stadtmitte014 Pem...
1  Stadtbezirk 2[2]     021 Flingern Süd022 Flingern Nord023 Düsseltal 

           District                                            Borough  \
0  Stadtbezirk ,[,]  , Altstadt, Carlstadt, Stadtmitte, Pempelfort,...   
1  Stadtbezirk ,[,]           , Flingern Süd, Flingern Nord, Düsseltal   

  district_numb  
0             1  
1             2  


In [4]:
### Get several District cells for each Borough
# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(', ')))

# calculate lengths of splits
lens = df['Borough'].str.split(', ').map(len)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'District': np.repeat(df['District'], lens),
                    'Borough': chainer(df['Borough']),
                    'district_numb': np.repeat(df['district_numb'], lens)})
# Strip empty cells ""
res = res[res['Borough'].str.strip().astype(bool)]

In [5]:
# Concatenate the District to include the Number for each District
res['Dis'] = res['District'].str[:11] 
res['District'] = res['Dis']+' '+res['district_numb']

# Finalize the DataFrame with Districts and Boroughs of Düsseldorf
df_boroughs = res[['District','Borough']]
df_boroughs = df_boroughs.reset_index(drop=True)


print('Scraping of Districts and Boroughs completed!\n')
print(df_boroughs.head(2))

Scraping of Districts and Boroughs completed!

        District    Borough
0  Stadtbezirk 1   Altstadt
1  Stadtbezirk 1  Carlstadt


#### 
## Getting the Coordinates for the city of Düsseldorf

In [6]:
# Getting the Coordinates for Düsseldorf

address = 'Düsseldorf, NRW'

geolocator = Nominatim(user_agent="duesseldorf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Düsseldorf are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Düsseldorf are 51.2254018, 6.7763137.


In [7]:
city = 'Düsseldorf, Germany'
boroughs  = df_boroughs['Borough']
coord = {'latitude':[],'longitude':[]}
for borough in boroughs:
    address = borough + ', ' + city
    location = geolocator.geocode(address)
    lat = location.latitude
    coord['latitude'].append(lat)
    lng = location.longitude
    coord['longitude'] .append(lng)
    
    print(address, lat, lng)

Altstadt, Düsseldorf, Germany 51.2259125 6.7735672
Carlstadt, Düsseldorf, Germany 51.2221416 6.7733942
Stadtmitte, Düsseldorf, Germany 51.2219385 6.7844229
Pempelfort, Düsseldorf, Germany 51.2396009 6.7796845
Derendorf, Düsseldorf, Germany 51.2445487 6.7922488
Golzheim, Düsseldorf, Germany 51.2507945 6.7599633
Flingern Süd, Düsseldorf, Germany 51.2210094 6.8100603
Flingern Nord, Düsseldorf, Germany 51.2313815 6.8132378
Düsseltal, Düsseldorf, Germany 51.2378412 6.812116
Friedrichstadt, Düsseldorf, Germany 51.2135645 6.7816997
Unterbilk, Düsseldorf, Germany 51.210055 6.7669651
Hafen, Düsseldorf, Germany 51.2170292 6.7335758
Hamm, Düsseldorf, Germany 51.2035725 6.7388087
Volmerswerth, Düsseldorf, Germany 51.1885784 6.7490097
Bilk, Düsseldorf, Germany 51.2027583 6.7851015
Oberbilk, Düsseldorf, Germany 51.2136887 6.8024279
Flehe, Düsseldorf, Germany 51.1922044 6.7717128
Oberkassel, Düsseldorf, Germany 51.2295816 6.7554498
Heerdt, Düsseldorf, Germany 51.2323927 6.7201136
Lörick, Düsseldorf, 

In [8]:
df_boroughs['latitude'] = coord['latitude']
df_boroughs['longitude'] = coord['longitude']
df_boroughs.head()

,District,Borough,latitude,longitude
0,Stadtbezirk 1,Altstadt,51.225912,6.773567
1,Stadtbezirk 1,Carlstadt,51.222142,6.773394
2,Stadtbezirk 1,Stadtmitte,51.221939,6.784423
3,Stadtbezirk 1,Pempelfort,51.239601,6.779685
4,Stadtbezirk 1,Derendorf,51.244549,6.792249


#### 
## Getting the number of habitants per borough

In [9]:
url_hab = 'http://www.citypopulation.de/en/germany/dusseldorf/admin/'
page = requests.get(url_hab)

    
# Parsing through the URL
soup = BeautifulSoup(page.content, 'html.parser')

In [10]:
pop_2018 = {'Borough':[], 'Population':[]}

for n in soup.find_all(itemprop="name"):
    n = n.decode()
    stuff = re.findall((r">(.*)\<"), n)
    if len(stuff) == 0:
        continue
    else:
        temp = []
        temp.append(stuff)
        for sublist in temp:
            for item in sublist:
                pop_2018['Borough'].append(item)
    
#Remove Continent and Country names + Removing old borough that was surpressed            
pop_2018['Borough'] = pop_2018['Borough'][3:45]+pop_2018['Borough'][46:] 


for p in soup.find_all('td', class_=['admin2', 'rpop prio1']):
        p = p.decode()
        stuff = re.findall((r'[0-9][0-9,.]+'),p)
        if len(stuff) == 0:
            continue
        else:
            temp = []
            temp.append(stuff)
            for sublist in temp:
                for item in sublist:
                    pop_2018['Population'].append(item)


pd_name = pd.Series(pop_2018['Borough'], dtype=object)
pd_pop = pd.Series(pop_2018['Population'], dtype=object)
population = pd.concat([pd_name, pd_pop], axis=1, sort=False)
population.columns = ['Borough', 'Population']
population.head()

,Borough,Population
0,Stadtbezirk 1,"85,914"
1,Altstadt,"2,404"
2,Carlstadt,"2,285"
3,Derendorf,"20,610"
4,Golzheim,"12,597"


# 
## Adding the Population to the DataFrame

In [11]:
df_dusseldorf = pd.merge(df_boroughs, population[['Borough','Population']],on='Borough')
df_dusseldorf['Population'] = df_dusseldorf['Population'].str.replace(",","").astype(int)
df_dusseldorf

,District,Borough,latitude,longitude,Population
0,Stadtbezirk 1,Altstadt,51.225912,6.773567,2404
1,Stadtbezirk 1,Carlstadt,51.222142,6.773394,2285
2,Stadtbezirk 1,Stadtmitte,51.221939,6.784423,14954
3,Stadtbezirk 1,Pempelfort,51.239601,6.779685,33064
4,Stadtbezirk 1,Derendorf,51.244549,6.792249,20610
5,Stadtbezirk 1,Golzheim,51.250794,6.759963,12597
6,Stadtbezirk 2,Flingern Süd,51.221009,6.810060,10344
7,Stadtbezirk 2,Flingern Nord,51.231381,6.813238,25111
8,Stadtbezirk 2,Düsseltal,51.237841,6.812116,28177
9,Stadtbezirk 3,Friedrichstadt,51.213564,6.781700,19883


# 
## Scrapping housing information

In [36]:
pnumber = 1 
housing = {'Address':[], 'Price':[], 'Rooms':[], 'Borough':[]}

while pnumber < 32:
    url_house = 'https://www.immobilienscout24.de/Suche/radius/wohnung-mieten?centerofsearchaddress=D%C3%BCsseldorf;;;1276010012;Nordrhein-Westfalen;&numberofrooms=1.0-&price=-3500.0&geocoordinates=51.23824;6.81513;3.0&sorting=4&pagenumber={}'.format(pnumber)
    headers = {"User-agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    page = requests.get(url_house, headers=headers)
    
    # Parsing through the URL
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Check if pages are being scraped
    #print('Page {} scraped'.format(pnumber))
    
    #Getting Address
    address  = soup.find_all('button')
    
    for a in address:
        a = a.decode()
        stuff = re.findall(("[^<>]+\w+[A-Za-z].\\s+Düsseldorf"), a)
        if len(stuff) == 0:
            continue
        else:
            temp = []
            temp.append(stuff)
            for sublist in temp:
                for item in sublist:
                    housing['Address'].append(item)
    
    #Getting price
    price  = soup.find_all('dd')
    
    for p in price:
        p = p.decode()
        stuff = re.findall(("[^<>]+\\s+€"),p)
        if len(stuff) == 0:
            continue
        else:        
            temp = []
            temp.append(stuff)
            for sublist in temp:
                for item in sublist:
                    housing['Price'].append(item)
    
    #Getting number of rooms
    rooms = soup.find_all(class_=['onlySmall'])
    
    for r in rooms:
        r = r.decode()
        stuff = re.findall(("[>]+[0-9]+[<]"), r)
        if len(stuff) == 0:
            continue
        else:   
            temp = []
            temp.append(stuff)
            for sublist in temp:
                for item in sublist:
                    housing['Rooms'].append(item)
    
    # Scrape the next page and keep running the While Loop
    pnumber = pnumber + 1

print("\nScraping completed.")    


Scraping completed.


In [37]:
# Get only the Borough from the Address Field - Not all houses have the full address
for address in housing['Address']:
    temp = address.split(", ")
    housing['Borough'].append(temp[-2])

pd_borough = pd.Series(housing['Borough'], dtype = object)
pd_price = pd.Series(housing['Price'], dtype = object)
pd_rooms = pd.Series(housing['Rooms'], dtype = object)

# Finishing cleaning the number of rooms
pd_rooms = pd_rooms.str[1]

# Concatenating the Series into a Pandas DataFrame
result = pd.concat([pd_borough, pd_price, pd_rooms], axis=1, sort=False)
result.columns = ['Borough', 'Price', 'Num_Rooms']
result['Price'] = result['Price'].map(lambda x: x.rstrip(' €'))


In [38]:
df_dusseldorf.index.values
df_dusseldorf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   District    50 non-null     object 
 1   Borough     50 non-null     object 
 2   latitude    50 non-null     float64
 3   longitude   50 non-null     float64
 4   Population  50 non-null     int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 2.3+ KB


In [39]:
result.index.values
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Borough    620 non-null    object
 1   Price      620 non-null    object
 2   Num_Rooms  560 non-null    object
dtypes: object(3)
memory usage: 14.7+ KB


In [30]:
df_dusseldorf.index.values
df_dusseldorf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   District    50 non-null     object 
 1   Borough     50 non-null     object 
 2   latitude    50 non-null     float64
 3   longitude   50 non-null     float64
 4   Population  50 non-null     int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 2.3+ KB


In [31]:
#df_group_one = df_dusseldorf.groupby(['drive-wheels'],as_index=False).mean()
#df_group_one